In [1]:
# https://github.com/YosefLab/velovi_reproducibility/blob/main/notebooks/estimation_comparison/simulated.ipynb

import os
import sys

import numpy as np
import pandas as pd
from scipy.stats import pearsonr
from sklearn.preprocessing import MinMaxScaler

import matplotlib.pyplot as plt
import seaborn as sns

import scvelo as scv

In [2]:
sns.reset_defaults()
sns.reset_orig()
scv.settings.set_figure_params('scvelo', dpi_save=400, dpi=80, transparent=True, fontsize=20, color_map='viridis')

In [3]:
def get_adata(n_obs, mu, R, C, n_vars=1000):
    cov = C.dot(C.T) * R

    alpha, beta, gamma = np.exp(np.random.multivariate_normal(mu, cov, size=n_vars).T)  # multivariate log-normal
    beta /= 3
    gamma /= 3

    # remove outliers
    idx = (alpha < np.percentile(alpha, 99)) & (beta < np.percentile(beta, 99)) & (gamma < np.percentile(gamma, 99))
    alpha = alpha[idx]
    beta = beta[idx]
    gamma = gamma[idx]
    n_vars = np.sum(idx)

    switches = np.random.uniform(.1, .5, size=n_vars)

    adata = scv.datasets.simulation(
        n_obs=n_obs,
        t_max=20,
        n_vars=n_vars,
        switches=switches,
        alpha=alpha,
        beta=beta,
        gamma=gamma,
        noise_level=.8
    )

    scv.pp.neighbors(adata)
    scv.tl.velocity(adata, mode='steady_state', use_raw=True)

    return adata

In [4]:
correlation_df = pd.DataFrame(columns=['correlation', 'method', 'n_obs', 'run'])

adatas = {}
bdatas = {}

mu = np.array([1, .2, .05])

R = np.array([[1., .2, .2],
              [.2, 1., .8],
              [.2, .8, 1.]])

C = np.array([.4, .4, .4])[:, None]
n_obs = 5000

In [5]:
adata = get_adata(n_obs=n_obs, mu=mu, R=R, C=C)

computing neighbors
    finished (0:00:09) --> added 
    'distances' and 'connectivities', weighted adjacency matrices (adata.obsp)
computing velocities
    finished (0:00:00) --> added 
    'velocity', velocity vectors for each individual cell (adata.layers)


In [6]:
adata

AnnData object with n_obs × n_vars = 5000 × 975
    obs: 'true_t'
    var: 'true_t_', 'true_alpha', 'true_beta', 'true_gamma', 'true_scaling', 'velocity_gamma', 'velocity_qreg_ratio', 'velocity_r2', 'velocity_genes'
    uns: 'pca', 'neighbors', 'velocity_params'
    obsm: 'X_pca'
    varm: 'PCs'
    layers: 'unspliced', 'spliced', 'velocity'
    obsp: 'distances', 'connectivities'

In [ ]:
run_id = 1
# Steady-state model
adata.var['true_ratio'] = np.array(adata.var['true_gamma'] / adata.var['true_beta'])
adata.var['steady_state_ratio'] = adata.var['velocity_gamma']
corr = pearsonr(adata.var['true_ratio'], adata.var['steady_state_ratio'])[0]
# Create a new DataFrame for the row you want to append
new_row = pd.DataFrame([{'correlation': corr, 'method': 'Steady-state', 'n_obs': n_obs, 'run': run_id}])
# Concatenate the new row to the existing DataFrame
correlation_df = pd.concat([correlation_df, new_row], ignore_index=True)

# EM model
scv.tl.recover_dynamics(
    adata, var_names=adata.var_names, use_raw=True, n_jobs=8
)
adata.var['em_ratio'] = adata.var['fit_gamma'] / adata.var['fit_beta']
corr = pearsonr(adata.var['true_ratio'], adata.var['em_ratio'])[0]
# Create a new DataFrame for the row you want to append
new_row = pd.DataFrame([{'correlation': corr, 'method': 'EM', 'n_obs': n_obs, 'run': run_id}])
# Concatenate the new row to the existing DataFrame
correlation_df = pd.concat([correlation_df, new_row], ignore_index=True)


recovering dynamics (using 8/20 cores)
or disable the progress bar using `show_progress_bar=False`.
    finished (0:00:39) --> added 
    'fit_pars', fitted parameters for splicing dynamics (adata.var)


In [18]:
correlation_df

,correlation,method,n_obs,run
0,0.771909,Steady-state,5000,1
1,0.952914,EM,5000,1


In [20]:
adata

AnnData object with n_obs × n_vars = 5000 × 975
    obs: 'true_t'
    var: 'true_t_', 'true_alpha', 'true_beta', 'true_gamma', 'true_scaling', 'velocity_gamma', 'velocity_qreg_ratio', 'velocity_r2', 'velocity_genes', 'true_ratio', 'steady_state_ratio', 'fit_alpha', 'fit_beta', 'fit_gamma', 'fit_t_', 'fit_scaling', 'fit_std_u', 'fit_std_s', 'fit_likelihood', 'fit_u0', 'fit_s0', 'fit_pval_steady', 'fit_steady_u', 'fit_steady_s', 'fit_variance', 'fit_alignment_scaling', 'em_ratio'
    uns: 'pca', 'neighbors', 'velocity_params', 'recover_dynamics'
    obsm: 'X_pca'
    varm: 'PCs', 'loss'
    layers: 'unspliced', 'spliced', 'velocity', 'fit_t', 'fit_tau', 'fit_tau_'
    obsp: 'distances', 'connectivities'